<img src="../docs/assets/JobShopLabLogo.svg" width="50" align="left" style="margin-right: 10px;">

# Getting Started with JobShopLab 



The **JobShopLab** is a Framework for solving real world JobShopScheduling scenarios using Reinforement Learning.


This Getting Started Guide provides some Examples and Explanations on how to use the Framework



## Table of Contents

- [Example of solving a ft06 instance with a random policy](#example)
- [Design Choices](#design_choices)
- [Configuring the Framework](#configuring-the-framework)
    - [Config File](#config-file)
    - [Dependency Injection](#dependency-injection)
- [Customizing](#customing)
- [Defining a Problem](#defining-a-problem)
    - [Spec Files](#spec-files)
    - [The Compiler](#the-compiler)
    - [DSL](#dsl)
    - [DSL as a string](#dsl-as-a-string)
- [Visualization](#visualization)
- [Defining Agents and Solving the Environment](#defining-agents-and-solving-the-environment)




## Installation

> **Note:**  
> Ensure you have the required Python version 3.12 or higher installed before proceeding.

To install JobShopLab, clone the repository and install it in editable mode using `pip`:



```bash
cd <desired_dir>
# ssh
git clone git@github.com:proto-lab-ro/jobshoplab.git
# or https
git clone https://github.com/proto-lab-ro/jobshoplab.git
# install python module in editable mode
pip install -e <repo_dir>
```


In [1]:
from jupyter_utils import change_to_jobshoplab, show_mermaid
change_to_jobshoplab()

Already in the desired directory: /home/felix/JobShopLab



## Example


the following example demonstrates how to solve an academic ft06 instance using a random policy.

In [2]:
# Solve a academic ft06 instance with a random policy

from jobshoplab import JobShopLabEnv, load_config
from pathlib import Path

config = load_config(config_path = Path("./data/config/getting_started_config.yaml"))
env = JobShopLabEnv(config=config)
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, truncated,terminated, info = env.step(action)
    done = truncated or terminated
env.render()

<div id='design_choices'/>
    
# Design Choices 

<img src="../docs/assets/FrameworkOverview.svg" width="600">

*JobShopLab is designed to be fully extensible and customizable.*


At its core is a state machine implemented in a functional programming style.<br>
The state machine takes inputs and produces outputs using an immutable data type object (interface).<br>

To address the need for various time mechanisms, observation spaces, and action spaces, a software layer called *Middleware* is used.<br> The Middleware sits between the Gym environment and the state machine, translating the Gym interface to the state machine interface.<br>



> **Note:**  
> Observation, reward, and action factories are injected into the Middleware to allow customization.<br> Their interfaces are represented by data type objects (dataclasses).



## Rendering


Rendering is performed via a dashboard built with Dash. <br>The dashboard displays a central Gantt chart and a table with all schedules. <br> Dashboards can be shown inline in a Jupyter Notebook or in a web browser.


> **Hint:**  
> Additional rendering utilities are available for debugging,<br> and a 3D simulation web app can also be accessed via the browser.


## Configuration Management


A critical aspect of maintaining a framework like JobShopLab and ensuring reproducible results is proper configuration management.


There are two types of configurations to consider:


### 1. Framework Config  


A YAML file containing configuration parameters that control the behavior of the framework.<br>
**Examples:** Setting the observation space, render modes, and truncation behavior.


### 2. Problem Instance Config  
A DSL file (a domain-specific language in YAML syntax) that defines the scheduling problem.  


## Configuring the Framework


The framework can be configured in two ways:


- Via a [config.yaml](#config-file) file
- Via [dependency injection](#dependency-injection)



> **Note**<br>
> A configuration file is always required, but it can be overridden using dependency injection.<br> While dependency injection is useful for testing new components quickly,<br> the recommended approach is to use configuration objects.

## Config File


Framework Configs are written in YAML syntax. <br>
The YAML gets parsed dynamically into a Python dataclass object.<br>


This allows:
 - Dot notation attribute access
 - Autocompletion
 - Type safety and validation


```yaml
# Example configuration file for JobShopLab

title: "Example Environment"
default_loglevel: "warning" # JobShopLab uses the logging module with a Logger obj for Verbosity. Set a default loglevel here


# Define all Dependencies for the environment here.. want to use a different observation? Simply change the ActionFactory name
env: 
    loglevel: "warning"
    observation_factory: "BinaryActionObservationFactory"
    reward_factory: "BinaryActionJsspReward"
    action_factory: "BinaryJobActionInterpreter"
    render_backend: "render_in_dashboard"
    middleware: "EventBasedBinaryActionMiddleware"
    max_time_fct: 2
    max_action_fct: 3

# For every Software Component there is a designated field 
compiler:
    loglevel: "warning"
    repo: "SpecRepository" # set to "YamlRepository" in order to use custom Problem Instances
    validator: "DummyValidator"
    manipulators: # want to randomize machine times? Set one or multiple Manipulators 
        - "DummyManipulator"
    dsl_repository:
        dir: "data/jssp_instances/dsl/default_instance.yaml" # yaml instance file to read
    spec_repository:
        dir: "data/jssp_instances/ft06" # academic spec file to read
state_machine:
    loglevel: "warning"

middleware:
    event_based_binary_action_middleware:
        loglevel: "warning"
        truncation_joker: 5 # 
        truncation_active: False

action_factory:
    binary_job_action_action_factory:
        loglevel: "warning"
    # want to add settings for a new action_factory (change the action space)
    # set a new field here with name of the class as lowercase 

observation_factory:
    binary_action_observation_factory:
        loglevel: "warning"
    
    

reward_factory:
    binary_action_jssp_reward:
        loglevel: "warning"
        sparse_bias: 1 # settings are passed as kwargs to the constructor of the Class
        dense_bias: 0.001
        truncation_bias: -1

render_backend:
    render_in_dashboard:
        loglevel: "warning"
        port: 8050
        debug: False
    simulation:
        json_dump_dir: "data/tmp/simulation_interface.json"
        port: 8051
        loglevel: "warning"
        bind_all: False
```



> **warning**<br>
> The configuration object is immutable. Any attempt to modify its attributes (as shown in the next example) will result in an error.

To load the configuration file and create an environment:


In [3]:
# load a config file
config = load_config(config_path=Path("./data/config/getting_started_config.yaml"))

# config is a object (immutable dataclass) atributtes can be accessed with dot notation
# a stub file (.pyi) is automaticly created when the config is loaded to ensure autocompletion
print(f"Dashboard Port: {config.render_backend.render_in_dashboard.port}") 

# config is immutable to prevent accidental changes and enforce consistency
try:
    config.render_backend.render_in_dashboard.port = 1000
except AttributeError as e:
    print(f"Error: {e}")

# create an environment with the loaded config
env = JobShopLabEnv(config=config)

Dashboard Port: 8050
Error: cannot assign to field 'port'


## ⏬ Dependency Injection

The environment allows you to pass dependencies directly as constructor arguments.<br>

The passed instances are constructed inside the environment, and additional arguments not included in the config file can be provided via partial application. <br>This supports dynamic instance creation (useful for hyperparameter optimization), rapid experiment implementation, and customization.

> **Note**
>   Dependency injection is particularly useful when you need to test new components or override default behaviors quickly.





Example using dependency injection with a dummy observation factory:

In [4]:
from jobshoplab.env.factories.observations import DummyObservationFactory
from functools import partial
config = load_config(config_path=Path("./data/config/getting_started_config.yaml"))

observation_factory = DummyObservationFactory # instead of using the observation factory from the config, 
# we will use a dummy observation factory that returns a random observation

observation_factory = partial(DummyObservationFactory, test_var="test_var") # we can pass additional arguments to the observation factory using partial application

env = JobShopLabEnv(config=config,observation_factory=observation_factory)
assert isinstance(env.state_simulator.observation_factory, DummyObservationFactory) # check that the observation factory is the one we passed
assert env.state_simulator.observation_factory.test_var == "test_var" # check that the argument was passed correctly

## ✍️ Customing

Want to introduce a new reward system?<br>

The framework provides base classes for every factory, enabling easy customization.<br> This applies to the observation factory and the action factory as well. Abstract base classes define their interfaces.


> **hint**
> Customizing these components allows you to tailor JobShopLab to fit your specific scheduling and reinforcement learning needs.

For example, you can define a custom reward factory as follows:

In [5]:
from jobshoplab.env.factories.rewards import RewardFactory
from jobshoplab.types import StateMachineResult


class CustomRewardFactory(RewardFactory): # inherit from the abstract base class for the desired factory
    # note: every ActionFactory takes 3 default arguments: loglevel, config and instance, additional arguments can be passed to the constructor
    def __init__(
        self, loglevel, config, instance, bias_a, bias_b, *args, **kwargs
    ):  # add args and kwargs to the constructor as placeholders
        self.test_var = super().__init__(
            loglevel, config, instance
        )  # call the parent constructor and pass the loglevel, config and instance
        self.bias_a, self.bias_b = bias_a, bias_b

    # the make method is called to create a reward (see the abstract base class RewardFactory)
    def make(self, state_result: StateMachineResult, terminated: bool, truncated: bool) -> float:
        if not terminated or truncated:
            return self.bias_a
        else:  # if done return a reward proportional to the time taken (makespan)
            return self.bias_b * state_result.state.time.time

    def __repr__(self) -> str:  # add a custom representation to the factory
        return f" CustomRewardFactory with bias_a: {self.bias_a}, bias_b: {self.bias_b}"



In [6]:

bias_a, bias_b = 0, 1
reward_factory = partial(
    CustomRewardFactory, bias_a=bias_a, bias_b=bias_b
)  # pass additional arguments to the constructor using partial application
env = JobShopLabEnv(config=config, reward_factory=reward_factory)
assert env.reward_factory.bias_a == bias_a  # argument was passed correctly and is accessible

## lets run a random env and track the rewards
done = False
rewards = []
while not done:
    action = env.action_space.sample()
    obs, reward, truncated, terminated, info = env.step(action)
    done = truncated or terminated
    rewards.append(reward)

print(f"Rewards: {rewards}")

Rewards: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 98]


# Defining a Problem

There are 3 ways of setting the Instance (Problem to solve)

In [7]:
show_mermaid("./docs/assets/instance.mmd")


1. **Spec File:** Academic JSSP problem definitions commonly found in the literature.
2. **DSL File:** A YAML file that defines the scheduling problem in a flexible way for real-world scenarios.
3. **DSL String:** Useful for Jupyter notebooks or for testing and debugging purposes.

### Spec Files

Spec files are academic JSSP problem instances as found in the literature.<br> Common instances are included in JobShopLab.<br> To use them, specify the SpecRepository in your configuration file:


```yaml
compiler:
  loglevel: *default_loglevel
  repo: "SpecRepository" # use the SpecRepository here
  validator: "DummyValidator"
  manipulators:
    - "DummyManipulator"
  dsl_repository:
    dir: "data/jssp_instances/dsl/default_instance.yaml"
  spec_repository:
    dir: "data/jssp_instances/ft06" # set the dir to the file here
```


Alternatively use **dependency injection:**

In [9]:
from jobshoplab.compiler.repos import SpecRepository
from jobshoplab.compiler import Compiler

repo = SpecRepository(dir=Path("data/jssp_instances/spec_files/ft06"),loglevel="warning",config=config)
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

### The Compiler


The compiler generates two interfaces used throughout JobShopLab:


- **Instance:** A dataclass object holding all the information about the problem.
- **(Initial) State:** A dataclass object representing the current state of the schedule. Since the state is time-dependent, <br>every state includes a time attribute.


> **note**<br>
>The compiler aggregates inputs from various sources (which can be overridden via dependency injection) and compiles a generic interface for the scheduling problem.


In [10]:
from jobshoplab.types import InstanceConfig, State
# Compiler has a compile method that returns the compiled instance and the initial state
instance , init_state = compiler.compile()

instance : InstanceConfig
init_state : State

# Some examples of accsessing artibs 


# the instance is a dataclass object witch is a representation for the Jssp Problem Interface.
# The instance is static and does not change
# note: the instance is immutable to prevent accidental changes and enforce consistency
print("Instance")
print("Some Machine ID:", instance.machines[0].id)
print("Operation Duration:", instance.instance.specification[0].operations[0].duration)

# the initial state is a dataclass object representing the initial state of all components
# the state is dynamic meaning that it changes as the environment is stepped its basicly a snapshot of the environment
# the state gets updated by the state simulator (state_machine)
# note: the state is immutable to prevent accidental changes and enforce consistency
print("\nState:")
print("Time:", init_state.time)
print("MachineState:",init_state.machines[0].state)

Instance
Some Machine ID: m-0
Operation Duration: DeterministicTimeConfig(time=1)

State:
Time: Time(time=0)
MachineState: MachineStateState.IDLE


### DSL


The primary purpose of JobShopLab is to represent real-world scheduling problems.<br> Therefore, the DSL (instance.yaml) provides a way to specify the problem itself.



**Specify the DslRepository in the config file:**
```yaml
compiler:
  loglevel: *default_loglevel
  repo: "DslRepository" # <- set the repo here
  validator: "DummyValidator"
  manipulators:
    - "DummyManipulator"
  dsl_repository:
    dir: "data/jssp_instances/dsl/default_instance.yaml" # <- set the filepath here
  spec_repository:
    dir: "data/jssp_instances/ft06"
```


**Alternatively** use dependency injection:

In [11]:
from jobshoplab.compiler.repos import DslRepository
from jobshoplab.compiler import Compiler

repo = DslRepository(dir=Path("data/jssp_instances/dsl/default_instance.yaml"),loglevel="warning",config=config)
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

### DSL as a string


An instance can also be defined inline as a string.<br> This approach is useful when working in Jupyter notebooks or for debugging and testing purposes.


> **Note**<br>
> Using a DSL string can simplify rapid prototyping or testing without the need for external YAML files.


In [15]:
dsl_str = """
title: InstanceConfig

# Example of a 6x6 Instance
# with AGVs

instance_config:
  description: "ft06 with AGVs" 
  instance:
    description: "6x6"
    specification: |
      (m0,t)|(m1,t)|(m2,t)|(m3,t)|(m4,t)|(m5,t)
      j0|(2,1) (0,3) (1,6) (3,7) (5,3) (4,6)
      j1|(1,8) (2,5) (4,10) (5,10) (0,10) (3,4)
      j2|(2,5) (3,4) (5,8) (0,9) (1,1) (4,7)
      j3|(1,5) (0,5) (2,5) (3,3) (4,8) (5,9)
      j4|(2,9) (1,3) (4,5) (5,4) (0,3) (3,1)
      j5|(1,3) (3,3) (5,9) (0,10) (4,4) (2,1)

    transport:
      type: "agv"
      amount: 6
  logistics: 
    specification: |
      m-0|m-1|m-2|m-3|m-4|m-5|in-buf|out-buf
      m-0|0 21 16 9 37 41 19 19
      m-1|21 0 13 15 17 23 8 8
      m-2|16 13 0 13 23 28 7 7
      m-3|9 15 13 0 31 35 14 14
      m-4|37 17 23 31 0 7 25 25
      m-5|41 23 28 35 7 0 24 24
      in-buf|19 8 7 14 25 24 0 0
      out-buf|19 8 7 14 25 24 0 0
      
init_state:
  t-0:
    location: m-0
  t-1:
    location: m-1
  t-2:
    location: m-2
  t-3:
    location: m-3
  t-4:
    location: m-4
  t-5:
    location: m-5
""" 

In [16]:
from jobshoplab.compiler.repos import DslStrRepository
from jobshoplab.compiler import Compiler

repo = DslStrRepository(dsl_str=dsl_str,loglevel="warning",config=config) # using the DslStrRepository here and passing the sting as an Argument
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)


> A **full example** and explanation of the DSL can be found in "data/jssp_instances/dsl/real_world_instance.yaml"

# 🖌️Visualization

In [17]:
show_mermaid("./docs/assets/rendering.mmd")

JobShopLab provides three main methods for visualizing an environment's state:


1. **Gantt Chart Dashboard:** A Dash web application that displays schedules on a timeline.
2. **CLI Table:** A debugging table rendered using the rich library.
3. **Simulation WebApp:** A 3D simulation using Three.js to render scenes of the schedules (coming soon!).



The default render mode can be configured in the config.yaml. For example:

```yaml

env:
  loglevel: *default_loglevel
  observation_factory: "BinaryActionObservationFactory"
  reward_factory: "BinaryActionJsspReward"
  action_factory: "BinaryJobActionInterpreter"
  render_backend: "render_in_dashboard" # <- set the render backend here
  middleware: "EventBasedBinaryActionMiddleware"

# Available Options and Settings
render_backend:
  render_in_dashboard:
    loglevel: *default_loglevel
    port: 8050
    debug: false

  simulation:
    json_dump_dir: "data/tmp/simulation_interface.json"
    port: 8051
    loglevel: *default_loglevel
    bind_all: false
  
  cli_table:
    loglevel: *default_loglevel

```


When calling ``env.render()``, you can pass a mode flag to select the render backend:


- **normal** (default): Uses the default backend from the configuration.
- **dashboard**: Displays the Dash Gantt chart.
- **simulation**: Activates the 3D simulation with Three.js.
- **debug**: Shows the rich CLI table.


Example visualization:

In [18]:
repo = DslStrRepository(dsl_str=dsl_str,loglevel="warning",config=config) # using the DslStrRepository here
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

# run with random actions
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, truncated, terminated, info = env.step(action)
    done = truncated or terminated
env.render()
# env.render(mode="simulation")
# env.render(mode="debug")

# 🤖 Defining Agents and Solving the Environemnt

For more details on defining agents and solving the environment using reinforcement learning algorithms, please refer to the jobshopagent repository.


Below is an example of training an agent using Stable Baselines3:

In [19]:
# train an agent using stable baselines3

from stable_baselines3 import PPO
from jobshoplab import JobShopLabEnv

env = JobShopLabEnv(config=config)
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 70.6     |
|    ep_rew_mean     | 0.794    |
| time/              |          |
|    fps             | 746      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
